In [1]:
import sys
sys.path.append('../')
from abel import *
#from abel.classes.stage.impl import stage_quasistatic_2d_radiation_reaction as rr
import numpy as np
import matplotlib.pyplot as plt
from abel.utilities.statistics import prct_clean, prct_clean2d
from matplotlib.animation import FuncAnimation
import time

initializing ocelot...


In [2]:
# define driver
driver = SourceBasic()
driver.charge = -2.7e10 * SI.e # [C]
driver.energy = 31.25e9 # [eV]
driver.rel_energy_spread = 0.01
driver.bunch_length = 50e-6 # [m]
driver.z_offset = 580e-6 # [m]
driver.emit_nx, driver.emit_ny = 10e-6, 10e-6 # [m rad]
driver.beta_x, driver.beta_y = 30e-3, 30e-3 # [m]
driver.num_particles = 10000
driver.jitter.x = 0 # [m] 2e-6
driver.jitter.y = 0 # [m] 100e-9
driver.jitter.t = 10e-15 # [m]

# define stage
stage = StageHipace()
stage.driver_source = driver
stage.length = 11 # [m]
stage.plasma_density = 2e21 # [m^-3]
stage.nom_energy_gain = 31.9e9 # [eV]
stage.radiation_reaction = True

# define first stage (half length)
first_stage = StageHipace()
first_stage.driver_source = driver
first_stage.length = stage.length/2 # [m]
first_stage.plasma_density = 2e21 # [m^-3]
first_stage.nom_energy_gain = stage.nom_energy_gain/2 # [eV]
first_stage.radiation_reaction = True

# define beam
source = SourceBasic()
source.charge = -1e10 * SI.e # [C]
source.energy = 5e9 # [eV]
source.rel_energy_spread = 0.01
source.bunch_length = 28e-6 # [m]
source.z_offset = 0e-6 # [m]
source.emit_nx, source.emit_ny = 160e-6, 0.56e-6 # [m rad]
source.beta_x = stage.matched_beta_function(source.energy)
source.beta_y = source.beta_x
source.num_particles = 1000
source.jitter.t = driver.jitter.t

# define interstage
interstage = InterstageBasic()
interstage.beta0 = lambda E: stage.matched_beta_function(E)
interstage.dipole_length = lambda E: 1 * np.sqrt(E/10e9) # [m(eV)]
interstage.dipole_field = 1 # [T]

# beam delivery system
bds = BeamDeliverySystemBasic()
bds.beta_x, bds.beta_y = 8e-3, 0.4e-3 # [m]
bds.bunch_length = 0.75 * bds.beta_y

# define linac
linac = Linac()
linac.source = source
linac.stage = stage
linac.first_stage = first_stage
linac.interstage = interstage
linac.num_stages = 16

In [3]:
start = time.process_time()

linac.run(f'linac_hipace_{int(source.energy/1e9)}GeV_{interstage.dipole_field}T_ex_{source.emit_nx*1e6}_jitterx_{driver.jitter.x}_jittery_{driver.jitter.y}_RR_{stage.radiation_reaction}', num_shots=1, overwrite=True, parallel=False);

print((time.process_time() - start)/60)

Tracking element 1 (s = 0.0 m, -1.60 nC, 5.0 GeV, SourceBasic, stage 0)
Running job 5096094
>> Starting (0:00)
>> Running (0:05)
>> Running (0:10)
>> Running (0:15)
>> Running (0:20)
>> Running (0:26)
>> Running (0:31)
>> Running (0:36)
>> Running (0:41)
>> Running (0:47)
>> Running (0:52)
>> Running (0:58)
>> Running (1:03)
>> Running (1:09)
>> Running (1:14)
>> Running (1:20)
>> Running (1:25)
>> Running (1:30)
>> Running (1:36)
>> Running (1:41)
>> Running (1:46)
>> Running (1:51)
>> Running (1:56)
>> Running (2:01)
>> Running (2:06)
>> Ending (2:07)
>> Ending (2:07)
>> Ending (2:07)
>> Done!
Tracking element 2 (s = 5.5 m, -1.60 nC, 13.9 GeV, StageHipace, stage 1)
Tracking element 3 (s = 12.4 m, -1.60 nC, 13.9 GeV, InterstageBasic, stage 1)
Running job 5096144
>> Starting (0:00)
>> Running (0:04)
>> Running (0:09)
>> Running (0:14)
>> Running (0:20)
>> Running (0:25)
>> Running (0:30)
>> Running (0:36)
>> Running (0:41)
>> Running (0:46)
>> Running (0:52)
>> Running (0:57)


KeyboardInterrupt: 

In [ ]:
linac.plot_survey()

# plot beam evolution
fig_9GeV = linac.plot_evolution(use_stage_nums=False)
#fig_9GeV.savefig(f'Plots_with_updated_params/{int(source.energy/1e9)}GeV_{int(source.emit_nx*1e6)}mmmrad_{int(source.emit_ny*1e6)}\
#mmmrad_{interstage.dipole_field}T_RR_{stage.enable_rr}_B_{interstage.dipole_field}T_deleted_low_z_values_10k.png')

linac.plot_waterfalls()